# Setup

In [1]:
# pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet
# pip install llama-index-readers-file pymupdf
# pip install llama-index-vector-stores-postgres
# pip install llama-index-embeddings-huggingface
# pip install llama-index-llms-llama-cpp

Sentence transformers setup

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [3]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/majanamyslowska/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:       

Postgres setup

In [4]:
import psycopg2

db_name = "vector_db"
host = "localhost"
password = "password"
port = "5432"
user = "maja2"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [5]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

# Ingestion Pipeline

In [6]:
# example with a pdf doc

# !mkdir data
# !wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

Load data

In [7]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [8]:
loader = PyMuPDFReader()
documents = loader.load(file_path="./data2/llama2.pdf")

Use a Text Splitter to Split Documents

In [9]:
from llama_index.core.node_parser import SentenceSplitter

In [10]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [11]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

Manually Construct Nodes from Text Chunks

In [12]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

Generate Embeddings for each Node

In [13]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

Load Nodes into a Vector Store

In [14]:
vector_store.add(nodes)

['1d72d12b-c8ec-499a-9688-f22ccad83efe',
 '93231e4a-3678-4a2e-abbd-bb7676988777',
 '30d9dbcf-2645-417c-bb1d-ee2593e465a9',
 '23c452a7-922c-4ca4-81ae-7e008886d20e',
 'ebabf1bc-1ea0-4048-a9b8-62e6a6137f1f',
 '4c950062-9bde-4a62-943b-2212916bcf24',
 '07efb5ce-dd39-42e8-85c1-fefffed598fe',
 '337d0f38-b76c-4b32-b249-0371249a1294',
 'f9d09d2a-c3a7-4efd-af5f-55b284e50775',
 '193e9b23-83cc-4f02-a86d-eda03bb639b6',
 'dd774cf3-1544-4162-b109-2dda2ea9efad',
 'c1204d04-07aa-479b-b9c2-f33a6501682f',
 '4753070d-a3f2-40c9-90d4-b2f69fcebe19',
 'da4be5d7-73ec-4ed2-8973-12ad4ab8c713',
 '5e1ced51-686e-49cf-bcb3-739dd28d9375',
 '29ae852a-a876-4cd4-8ccd-9b7cd2aae042',
 '35257d93-05f8-4641-aa6a-4dd8d3af0009',
 '222654b3-70b9-4db5-81a1-5890d856639c',
 '758a051f-4930-4ae1-b373-130c7efc03ff',
 'eee1f5e5-9ffa-4a60-a4f6-5f8c2c81d7de',
 '594cfff2-7291-49d7-961f-71392c3a55f8',
 'cdf13853-f0d8-4b04-be06-e350c6f3685f',
 '61dde9a4-9c85-4077-b24d-5c3e35ca1ab9',
 '1246aef2-8d9f-42bc-803a-001dcc3d58a9',
 'ee726b0e-c902-

# Retrieval Pipeline

In [15]:
query_str = "Can you tell me about the key concepts for safety finetuning"

Generate a Query Embedding

In [16]:
query_embedding = embed_model.get_query_embedding(query_str)

Query the Vector Database

In [17]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [18]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary

Parse Result into a Set of Nodes

In [19]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

Put into a Retriever

In [20]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [21]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

# RetrieverQueryEngine Response

In [22]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [23]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)

In [ ]:
print(str(response))

In [ ]:
print(response.source_nodes[0].get_content())